In [3]:
#Webscraping imports
from bs4 import BeautifulSoup
import requests

#Data Analysis/Wrangling imports
import pandas as pd
import numpy as np

#Data Viz imports
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [4]:
BASE_URL = "https://www.metacritic.com/browse/games/score/metascore/all/all"

# Need a header, so that the Website didn't think, we are a bot
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

In [ ]:

# Instantiate data list to store dictionaries with information on games in it.
data = []

#Manually checked, there are 195 pages (numbered 0 to 194 for metacritic) as of 2022/06/25
for i in range(195):
    
    #Timing how long each loop takes.
    start_time = time.time()
    
    # Use request library to call the webpage with headers as headers
    page = requests.get(BASE_URL,
                        params={'page': i},
                        headers = headers)

    # Create instance of BeautifulSoup with the html content of the page for scraping
    soup = BeautifulSoup(page.content, "html.parser")
    
    # Collect all containers in games based on class 'clamp-summary-wrap'
    games = soup.find_all('td', class_='clamp-summary-wrap')

    for game in games:

        #Instatiate a new dictionary for each game to store relevant information
        game_dict = dict()

        #Extracting the rank
        try:
            game_dict['rank'] = int(game.find('span', class_='title numbered').text.strip().strip('.'))
        except:
            game_dict['rank'] = np.nan

        #Extracting the title of the game
        try:
            game_dict['title'] = game.find('a', class_='title').text
        except:
            game_dict['title'] = np.nan

        #Extracting platform/console game has been released on
        try:
            game_dict['platform'] = game.find('div', class_='platform').find('span', class_='data').text.strip()
        except:
            game_dict['platform'] = np.nan

        #Release date
        try:
            game_dict['release_date'] = game.find('div', class_='clamp-details').find_all('span')[2].text        
        except:
            game_dict['release_date'] = np.nan

        #Summary of the game
        try:
            game_dict['summary'] = game.find('div', class_='summary').text.strip()            
        except:
            game_dict['summary'] = np.nan

        #Metacritic Score
        try:
            game_dict['metacritic_score'] = int(game.find('div', class_='metascore_w large game positive').text)
        except:
            game_dict['metacritic_score'] = np.nan

        #User Score
        try:
            game_dict['user_score'] = float(game.find('div', class_='metascore_w user large game positive').text)
        except:
            game_dict['user_score'] = np.nan

        data.append(game_dict)
    
    print(f'Scraped page {i+1} out of 195 in {round(time.time()-start_time, 2)} secs.')

Scraped page 1 out of 195 in 1.52 secs.
Scraped page 2 out of 195 in 1.48 secs.
Scraped page 3 out of 195 in 1.56 secs.
Scraped page 4 out of 195 in 1.63 secs.
Scraped page 5 out of 195 in 1.58 secs.
Scraped page 6 out of 195 in 3.9 secs.
Scraped page 7 out of 195 in 4.04 secs.
Scraped page 8 out of 195 in 4.47 secs.
Scraped page 9 out of 195 in 4.46 secs.
Scraped page 10 out of 195 in 1.6 secs.
Scraped page 11 out of 195 in 4.23 secs.
Scraped page 12 out of 195 in 3.81 secs.
Scraped page 13 out of 195 in 4.1 secs.
Scraped page 14 out of 195 in 3.71 secs.
Scraped page 15 out of 195 in 4.25 secs.
Scraped page 16 out of 195 in 4.17 secs.
Scraped page 17 out of 195 in 4.21 secs.
Scraped page 18 out of 195 in 4.25 secs.
Scraped page 19 out of 195 in 4.33 secs.
Scraped page 20 out of 195 in 1.43 secs.
Scraped page 21 out of 195 in 1.48 secs.
Scraped page 22 out of 195 in 4.13 secs.
Scraped page 23 out of 195 in 4.36 secs.
Scraped page 24 out of 195 in 4.28 secs.
Scraped page 25 out of 195 i

In [ ]:
games_df = pd.DataFrame(data)

In [ ]:
#Checking datatypes for columns
games_df.info()

In [ ]:
#Release date is not in a datetime format. We need to convert it to the correct datatype

games_df['release_date'] = pd.to_datetime(games_df['release_date'])

In [ ]:
games_df.info()

In [ ]:
games_df.platform.unique()

In [ ]:
top1000 = games_df.loc[:1000,:]

In [ ]:
#Checking the platforms for top 1000 games

plt.figure(figsize=(10,10))
plt.xticks(rotation=90)
sns.histplot(top1000.platform)

In [ ]:
#Checking which year the top 1000 games were released in
sns.histplot(top1000.release_date.dt.year)